In [75]:
import requests

response = requests.get('https://api.github.com/repositories',
                        headers = {'Accept': 'application/vnd.github+json'})
print(response.status_code)

403


In [76]:
#response에 대한 요소 확인
print(f"인코딩 : {response.encoding}")
print(f"콘텐츠 타임 : {response.headers['Content-Type']}")
print(f"서버 : {response.headers['server']}")


인코딩 : utf-8
콘텐츠 타임 : application/json; charset=utf-8
서버 : Varnish


In [77]:
import json
print(json.dumps(response.json()[0], indent=2)[:200])

KeyError: 0

In [6]:
response.headers

{'Date': 'Mon, 27 Feb 2023 06:59:39 GMT', 'Server': 'Varnish', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'deny', 'X-XSS-Protection': '1; mode=block', 'Content-Security-Policy': "default-src 'none'; style-src 'unsafe-inline'", 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, X-RateLimit-Used, X-RateLimit-Resource, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, Deprecation, Sunset', 'Content-Type': 'application/json; charset=utf-8', 'Referrer-Policy': 'origin-when-cross-origin, strict-origin-when-cross-origin', 'X-GitHub-Media-Type': 'github.v3; format=json', 'X-RateLimit-Limit': '60', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1677481638', 'X-RateLimit-Resource': 'core', 'X-RateLimit-Used': '60', 'Content-Length': '280', 'X-GitH

In [27]:
response = requests.get('https://api.github.com/search/repositories',
                        params={'q':'data_science+language:python'},
                        headers = {'Accept': 'application/vnd.github+json'})
print(response.status_code)

200


In [28]:
response = requests.get('https://api.github.com/search/repositories',
                        params={'q':'data_science+language:python'},
                        headers = {'Accept': 'application/vnd.github.text-match+json'})
print(response.status_code)

200


In [29]:
#json파일 딕셔너리형으로 변경하기
import json
a =response.json()

In [30]:
response.json().keys()

dict_keys(['total_count', 'incomplete_results', 'items'])

In [31]:
response.json()['items'][0]['text_matches']

[{'object_url': 'https://api.github.com/repositories/26382146',
  'object_type': 'Repository',
  'property': 'description',
  'fragment': 'code for Data Science From Scratch book',
  'matches': [{'text': 'Data Science', 'indices': [9, 21]}]},
 {'object_url': 'https://api.github.com/repositories/26382146',
  'object_type': 'Repository',
  'property': 'name',
  'fragment': 'data-science-from-scratch',
  'matches': [{'text': 'data', 'indices': [0, 4]}]}]

In [32]:
response.json().keys()

dict_keys(['total_count', 'incomplete_results', 'items'])

In [34]:
len(response.json()['items'])

30

In [56]:
for item in response.json()['items']:
    print(item['name']+' : repository   '+ 
          item['text_matches'][0]['property']+'-\"'+
          item['text_matches'][0]['fragment']+'\" matched with   '+
          item['text_matches'][0]['matches'][0]['text']
         )

data-science-from-scratch : repository   description-"code for Data Science From Scratch book" matched with   Data Science
PySyft : repository   description-"data science on data without acquiring a copy" matched with   data science
data-science-blogs : repository   description-"A curated list of data science blogs" matched with   data science
galaxy : repository   description-"Data intensive science for everyone." matched with   Data
DataCamp : repository   description-"DataCamp data-science courses" matched with   data
data-scientist-roadmap : repository   description-"Toturials coming with the "data science roadmap" picture." matched with   data science
dsp : repository   description-"data science preparation" matched with   data science
cookiecutter-data-science : repository   description-"A logical, reasonably standardized, but flexible project structure for doing and sharing data science work." matched with   data science
Kaggler : repository   description-"Code for Kaggle Data S

In [59]:

response = requests.get('https://api.github.com/repos/pytorch/pytorch/issues',
                       headers={'Accept': 'application/vnd.github.text-match+json'})
print('Response Code', response.status_code)
print('Number of comments', len(response.json()))

Response Code 200
Number of comments 30


In [60]:
response.links

{'next': {'url': 'https://api.github.com/repositories/65600975/issues?page=2',
  'rel': 'next'},
 'last': {'url': 'https://api.github.com/repositories/65600975/issues?page=371',
  'rel': 'last'}}

In [61]:
def get_all_pages(url,params=None, headers=None):
    output_json=[]
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None:
                output_json += get_all_pages(next_url,params,headers)
    return output_json

In [63]:
import pandas as pd
out = get_all_pages(
    "https://api.github.com/repos/pytorch/pytorch/issues/comments",
    params={
        'since':'2023-01-01T10:00:012',
        'sorted': 'created',
        'direction':'desc'
    },
    headers={'Accept': 'application/vnd.github+json'}
)
df = pd.DataFrame(out)
print(df['body'].count())
df[['id','created_at','body']].sample(1)

1740


,id,created_at,body
1141,1441222871,2023-02-23T05:04:02Z,@pytorchbot merge


In [67]:
response = requests.head('https://api.github.com/repos/pytorch/pytorch/issues/comments')
print('X-Ratelimit-Limit : ', response.headers['X-Ratelimit-Limit'])
print('X-Ratelimit-Remaining : ',response.headers['X-Ratelimit-Remaining'])

import datetime
print('Rate Limit reset at : ', datetime.datetime.fromtimestamp(int(response.headers['X-RateLimit-Reset'])).strftime('%c'))

X-Ratelimit-Limit :  60
X-Ratelimit-Remaining :  0
Rate Limit reset at :  Mon Feb 27 18:18:39 2023


In [70]:
from datetime import datetime
import time

def handle_rate_limits(response):
    now=datetime.now()
    reset_time = datetime.fromtimestamp(int(response.headers['X-RateLimit-Reset']))
    remaining_requests = response.headers['X-RateLimit-Remaining']
    remaining_time = (reset_time - now).total_seconds()
    intervals = remaining_time / (1.0 + int(remaining_requests))
    
    print('sleeping for', intervals)
    time.sleep(intervals)
    return True


In [73]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

retry_strategy = Retry(
    total = 5,
    status_forcelist=[500,503,504],
    backoff_factor=1
)

retry_adapter = HTTPAdapter(max_retries=retry_strategy)

http = requests.Session()
http.mount("https://", retry_adapter)

response = http.get('https://api.github.com/search/repositories',
                   params = {'q':'data_science+language:python'})

for item in response.json()['items'][:5]:
    print(item['name'])

data-science-from-scratch
PySyft
data-science-blogs
galaxy
DataCamp
